# FULL_Modelling — M4–M7 Spatial Pipeline 
#### Run matrix: 3 exposures × 2 samples × 2 outcomes = 12 runs per model block

In [52]:
import os
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import pandas as pd
import numpy as np
from scipy import sparse

# IV / OLS
import statsmodels.api as sm
from linearmodels.iv import IV2SLS
# OLSResults unused; sm.OLS().fit() used for SLX/SDM first stage

# Spatial
import libpysal
from libpysal.weights import KNN
from libpysal.weights.spatial_lag import lag_spatial
from esda import Moran, Moran_Local
import spreg

# Geo / viz
import geopandas as gpd
import matplotlib.pyplot as plt

# Paths (match Stata ROOT)
ROOT = os.getcwd()
if not os.path.exists(os.path.join(ROOT, "Output")):
    ROOT = os.path.dirname(ROOT)

PATH_COAST = os.path.join(ROOT, "Output", "tsls",  "first_iv2sls_pesisir", "podes_vill_spbun_pca_iv.dta")
PATH_ALL   = os.path.join(ROOT, "Output", "tsls", "first_iv2sls", "podes_vill_spbun_pca_iv.dta")
PATH_TSLS  = os.path.join(ROOT, "Output", "tsls", "village_values")
BASEMAP_PATH = os.path.join(ROOT, "Survey_Map", "id.json")
# Load basemap as GeoDataFrame so it can be plotted as a layer (fix: map layer on basemap)
BASEMAP = gpd.read_file(BASEMAP_PATH) if os.path.exists(BASEMAP_PATH) else None

OUT_DIRS = [
    "Output", "Output/tsls",
    "Output/tsls/tables", "Output/tsls/figures", "Output/tsls/maps",
    "Output/models", "Output/logs",
]
for d in OUT_DIRS:
    os.makedirs(os.path.join(ROOT, d), exist_ok=True)

def out_path(*parts):
    return os.path.join(ROOT, "Output", "tsls", *parts)

print("ROOT:", ROOT)
print("COAST exists:", os.path.exists(PATH_COAST))
print("ALL exists:", os.path.exists(PATH_ALL))

ROOT: /Users/athamawardi/Desktop/Research-Projects/PSE_Pertamina
COAST exists: True
ALL exists: True


In [53]:
# --- Global config (single source of truth) ---
SAMPLES   = ["COAST", "ALL"]
EXPOSURES = ["A_v_log", "Ker5_w1", "Ker10_w1"]
OUTCOMES  = ["lnpov_z", "Wbasic_z"]

FE_KEY    = "kab"
CLUSTER_KEY = "kab"  # cluster-robust SE use FE_KEY (kab) in sm.OLS
INSTRUMENTS = ["Z_pel", "Z_tbbm", "Z_tpi"]
BASELINE_SPEC = "IVH_UTIL_NB"  # matches spec in village_values CSVs (was IVH_BASE — not present in data)
W_TYPE = "knn"  # informational; build_W uses K_NEIGHBORS and W_TRANSFORM
K_NEIGHBORS = 8
W_TRANSFORM = "R"

# Categorical controls (Stata X_S_base_cat; use raw names; _c if present in data)
X_CAT_NAMES = ["r308b1d", "r308b1e", "r309a", "r309b", "r309d", "r309e", "r310", "r403a", "r403c1", "r403c2"]
# PCA patterns: we'll select from dataframe (r509c dropped per plan)
X_PCA_PATTERNS = ["r701", "r711"]  # cols like r701*_b, r711*_b; r509c* excluded

In [54]:
# --- Load analysis datasets and build control set X ---
def load_analysis_data():
    coast = pd.read_stata(PATH_COAST)
    all_df = pd.read_stata(PATH_ALL)
    # Standardize types
    for df in [coast, all_df]:
        if "iddesa" in df.columns:
            df["iddesa"] = df["iddesa"].astype(str).str.strip()
        # Keep kab as-is (often category in .dta); do not convert to numeric (loses all rows)
        # if "kab" in df.columns: df["kab"] = pd.to_numeric(...)
        for c in ["lat_v", "lon_v"]:
            if c in df.columns:
                df[c] = pd.to_numeric(df[c], errors="coerce")
    return {"COAST": coast, "ALL": all_df}

def build_X(df):
    """Build X = X_CAT + X_PCA from available columns. Stata-aligned."""
    cat = [c for c in X_CAT_NAMES if c in df.columns]
    pca = [c for c in df.columns if c.endswith("_b") and any(c.startswith(p) for p in X_PCA_PATTERNS)]
    return cat + sorted(pca)

DATASETS = load_analysis_data()
# X_COLS: use only columns present in BOTH COAST and ALL (r509c dropped — not in X_PCA_PATTERNS)
x_all = build_X(DATASETS["ALL"])
X_COLS = [c for c in x_all if c in DATASETS["COAST"].columns]
print("X columns (%d):" % len(X_COLS), X_COLS[:15], "..." if len(X_COLS) > 15 else "")
print("r509c excluded from X (SDM and all models use X_COLS).")
print("COAST rows:", len(DATASETS["COAST"]), "| ALL rows:", len(DATASETS["ALL"]))

X columns (47): ['r308b1d', 'r308b1e', 'r309a', 'r309b', 'r309d', 'r309e', 'r310', 'r403a', 'r403c1', 'r403c2', 'r701ak3_b', 'r701ak4_b', 'r701bk3_b', 'r701bk4_b', 'r701ck4_b'] ...
r509c excluded from X (SDM and all models use X_COLS).
COAST rows: 12968 | ALL rows: 84276


In [55]:
# --- Load Stata baseline village_values for M4 residual Moran ---
# Files: village_values_long_v1/v2 (A_v_log), Ker5_*, Ker10_*
def load_baseline_residuals():
    out = {}
    # A_v_log
    for v in ["v1", "v2"]:
        p = os.path.join(PATH_TSLS, f"village_values_long_{v}.csv")
        if os.path.exists(p):
            df = pd.read_csv(p)
            df["iddesa"] = df["iddesa"].astype(str).str.strip()
            out[("A_v_log", v)] = df
    # Ker5
    for v in ["v1", "v2"]:
        p = os.path.join(PATH_TSLS, f"Ker5_village_values_long_{v}.csv")
        if os.path.exists(p):
            df = pd.read_csv(p)
            df["iddesa"] = df["iddesa"].astype(str).str.strip()
            out[("Ker5_w1", v)] = df
    # Ker10
    for v in ["v1", "v2"]:
        p = os.path.join(PATH_TSLS, f"Ker10_village_values_long_{v}.csv")
        if os.path.exists(p):
            df = pd.read_csv(p)
            df["iddesa"] = df["iddesa"].astype(str).str.strip()
            out[("Ker10_w1", v)] = df
    return out

BASELINE_VALUES = load_baseline_residuals()
print("Loaded baseline files:", list(BASELINE_VALUES.keys()))
# Use v1 for baseline uhat; filter by sample, outcome, spec
def get_baseline_uhat(exposure, sample, outcome, spec=BASELINE_SPEC):
    key = (exposure, "v1")
    if key not in BASELINE_VALUES:
        return None
    df = BASELINE_VALUES[key]
    m = (df["sample"] == sample) & (df["outcome"] == outcome) & (df["spec"] == spec)
    out = df.loc[m, ["iddesa", "uhat", "W_hat"]].copy()
    if len(out) == 0:
        m2 = (df["sample"] == sample) & (df["outcome"] == outcome)
        if m2.any():
            out = df.loc[m2, ["iddesa", "uhat", "W_hat"]].drop_duplicates(subset=["iddesa"])
    return out if len(out) > 0 else None

Loaded baseline files: [('A_v_log', 'v1'), ('A_v_log', 'v2'), ('Ker5_w1', 'v1'), ('Ker5_w1', 'v2'), ('Ker10_w1', 'v1'), ('Ker10_w1', 'v2')]


In [56]:
# --- Run dataframe: complete cases per (sample, exposure, outcome) ---
def build_run_df(sample, exposure, outcome):
    df = DATASETS[sample].copy()
    y = outcome
    A = exposure
    req = ["iddesa", FE_KEY, "lat_v", "lon_v", y, A] + X_COLS + INSTRUMENTS
    missing = [c for c in req if c not in df.columns]
    if missing:
        return None, {"error": "missing_cols", "cols": missing}
    run = df[req].dropna(how="any").reset_index(drop=True)
    run = run.rename(columns={y: "y", A: "A"})
    # Merge baseline uhat if available
    uhat_df = get_baseline_uhat(exposure, sample, outcome)
    if uhat_df is not None and len(uhat_df) > 0:
        run = run.merge(uhat_df[["iddesa", "uhat", "W_hat"]], on="iddesa", how="left")
    else:
        run["uhat"] = np.nan
        run["W_hat"] = np.nan
    manifest = {
        "sample": sample, "exposure": exposure, "outcome": outcome,
        "N_raw": len(df), "N_used": len(run),
    }
    return run, manifest

In [57]:
# --- Spatial weights W (kNN, row-standardized) per sample ---
def build_W(df):
    """df must have lat_v, lon_v; drop missing coords and duplicates for W."""
    g = df[["lat_v", "lon_v"]].dropna(how="any")
    g = g.drop_duplicates()
    coords = g[["lon_v", "lat_v"]].values
    W = KNN.from_array(coords, k=min(K_NEIGHBORS, len(coords) - 1))
    W.transform = W_TRANSFORM
    return W, g.index if hasattr(g, "index") else np.arange(len(g))

# Build W per sample (reserved; run-level W uses build_W_for_run below)
W_by_sample = {}
for sample in SAMPLES:
    df = DATASETS[sample]
    g = df[["lat_v", "lon_v"]].dropna(how="any").drop_duplicates()
    if len(g) < 2:
        W_by_sample[sample] = None
        continue
    coords = df.loc[g.index][["lon_v", "lat_v"]].values
    W = KNN.from_array(coords, k=min(K_NEIGHBORS, len(coords) - 1))
    W.transform = W_TRANSFORM
    W_by_sample[sample] = (W, g.index.tolist())

# For each run we need W aligned to run_df row order: rebuild W on run_df coords
def build_W_for_run(run_df):
    coords = run_df[["lon_v", "lat_v"]].values
    # Add tiny jitter to break ties from duplicate coordinates (prevents zero-distance KNN distortion)
    rng = np.random.RandomState(42)
    jitter = rng.normal(0, 1e-8, coords.shape)
    coords = coords + jitter
    k = min(K_NEIGHBORS, len(coords) - 1)
    if k < 1:
        return None
    W = KNN.from_array(coords, k=k)
    W.transform = W_TRANSFORM
    return W

In [58]:
# Preview pesisir (COAST) data
DATASETS["COAST"].head()

,iddesa,nama_prov,nama_kab,nama_kec,nama_desa,r604a,r604b,r604c,r604d,r604e,...,A_v_inv,D_pel_iv,D_tbbm_iv,D_tpi_iv,Z_pel,Z_tbbm,Z_tpi,sd_Z_pel,sd_Z_tbbm,sd_Z_tpi
0,1101010001,ACEH,SIMEULUE,TEUPAH SELATAN,LATIUNG,2,2,1,1,1,...,0.007692,18.700657,266.123706,94.590363,-2.980652,-5.587712,-4.560072,0.898514,0.056373,0.757342
1,1101010002,ACEH,SIMEULUE,TEUPAH SELATAN,LABUHAN BAJAU,2,2,2,2,2,...,0.007995,18.891550,261.118702,96.087853,-2.990295,-5.568797,-4.575616,0.898514,0.056373,0.757342
2,1101010003,ACEH,SIMEULUE,TEUPAH SELATAN,SUAK LAMATAN,2,2,2,2,2,...,0.007276,17.116508,275.192802,87.756199,-2.896824,-5.621099,-4.485893,0.898514,0.056373,0.757342
3,1101010004,ACEH,SIMEULUE,TEUPAH SELATAN,ANA AO,2,2,2,2,2,...,0.008168,12.603267,260.692994,90.080357,-2.610310,-5.567172,-4.511742,0.898514,0.056373,0.757342
4,1101010005,ACEH,SIMEULUE,TEUPAH SELATAN,LATALING,2,2,2,2,2,...,0.008221,11.319306,260.404786,88.743123,-2.511168,-5.566070,-4.496951,0.898514,0.056373,0.757342


## M4 — Diagnostic spatial autocorrelation (Moran's I on outcome and baseline residuals)

In [59]:
# M4.1 Moran's I on outcome (per sample × outcome)
# M4.2 Moran's I on baseline residuals (per sample × exposure × outcome)
moran_rows = []
permutations = 999

for sample in SAMPLES:
    df = DATASETS[sample]
    for outcome in OUTCOMES:
        ycol = outcome
        if ycol not in df.columns:
            continue
        run, _ = build_run_df(sample, EXPOSURES[0], outcome)  # any exposure for y-only
        if run is None or len(run) < 10:
            continue
        W = build_W_for_run(run)
        if W is None:
            continue
        y = run["y"].values
        mi = Moran(y, W, permutations=permutations)
        moran_rows.append({
            "sample": sample, "outcome": outcome, "exposure": "—",
            "target": "y", "moran_i": mi.I, "p_value": mi.p_sim, "z_score": getattr(mi, "z_sim", np.nan),
        })

for sample in SAMPLES:
    for exposure in EXPOSURES:
        for outcome in OUTCOMES:
            run, _ = build_run_df(sample, exposure, outcome)
            if run is None or run["uhat"].notna().sum() < 10:
                continue
            run = run.dropna(subset=["uhat"])
            W = build_W_for_run(run)
            if W is None:
                continue
            u = run["uhat"].values
            mi = Moran(u, W, permutations=permutations)
            moran_rows.append({
                "sample": sample, "outcome": outcome, "exposure": exposure,
                "target": "uhat_baseline", "moran_i": mi.I, "p_value": mi.p_sim, "z_score": getattr(mi, "z_sim", np.nan),
            })

moran_df = pd.DataFrame(moran_rows)
moran_df.to_csv(out_path("tables", "moran_results.csv"), index=False)
print(moran_df.to_string())

   sample   outcome  exposure         target   moran_i  p_value     z_score
0   COAST   lnpov_z         —              y  0.599305    0.001  141.885489
1   COAST  Wbasic_z         —              y  0.838022    0.001  196.673818
2     ALL   lnpov_z         —              y  0.655732    0.001  410.885337
3     ALL  Wbasic_z         —              y  0.862522    0.001  520.444103
4   COAST   lnpov_z   A_v_log  uhat_baseline  0.198883    0.001   48.563574
5   COAST  Wbasic_z   A_v_log  uhat_baseline  0.214338    0.001   49.982059
6   COAST   lnpov_z   Ker5_w1  uhat_baseline  0.217904    0.001   52.884868
7   COAST  Wbasic_z   Ker5_w1  uhat_baseline  0.227698    0.001   53.161837
8   COAST   lnpov_z  Ker10_w1  uhat_baseline  0.221004    0.001   51.456239
9   COAST  Wbasic_z  Ker10_w1  uhat_baseline  0.235327    0.001   54.143538
10    ALL   lnpov_z   A_v_log  uhat_baseline  0.217244    0.001  133.880503
11    ALL  Wbasic_z   A_v_log  uhat_baseline  0.268066    0.001  163.302409
12    ALL   

## M5 — SLX (Spatial Lag of X): y = β·A + θ·(W·A) + Γ·X + ε (FE: demean by kab)

In [ ]:
# M5: SLX with FE (demean by kab), cluster-robust SE
def demean_by(df, key, cols):
    out = df.copy()
    for c in cols:
        if c in out.columns and pd.api.types.is_numeric_dtype(out[c]):
            out[c] = out[c] - out.groupby(key, observed=True)[c].transform("mean")
    return out

def run_slx(run_df, W, X_cols):
    """SLX on demeaned y, A, WA, X. OLS with cluster-robust SE (kab)."""
    run = run_df.copy()
    run["WA"] = lag_spatial(W, run["A"].values)
    to_demean = ["y", "A", "WA"] + [c for c in X_cols if c in run.columns]
    run = demean_by(run, FE_KEY, to_demean)
    exog = ["A", "WA"] + [c for c in X_cols if c in run.columns and run[c].notna().all() and pd.api.types.is_numeric_dtype(run[c])]
    X = sm.add_constant(run[exog].astype(np.float64), has_constant="add")
    y = run["y"].astype(np.float64)
    groups = run[FE_KEY].astype(str) if run[FE_KEY].dtype.name == "category" else run[FE_KEY]
    mod = sm.OLS(y, X).fit(cov_type="cluster", cov_kwds={"groups": groups})
    resid = run["y"].values - mod.predict(X)
    return mod, resid, exog

slx_rows = []
slx_moran_rows = []
for sample in SAMPLES:
    for exposure in EXPOSURES:
        for outcome in OUTCOMES:
            run, _ = build_run_df(sample, exposure, outcome)
            if run is None or len(run) < 20:
                continue
            W = build_W_for_run(run)
            if W is None:
                continue
            try:
                mod, resid, exog = run_slx(run, W, X_COLS)
                for p in exog:
                    if p in mod.params.index:
                        slx_rows.append({
                            "sample": sample, "exposure": exposure, "outcome": outcome,
                            "term": p, "coef": mod.params[p], "se": mod.bse[p], "pvalue": mod.pvalues[p],
                        })
                # Post-fit Moran on SLX residuals
                mi = Moran(resid, W, permutations=999)
                slx_moran_rows.append({
                    "sample": sample, "exposure": exposure, "outcome": outcome,
                    "moran_i": mi.I, "p_value": mi.p_sim,
                })
            except Exception as e:
                slx_rows.append({"sample": sample, "exposure": exposure, "outcome": outcome, "term": "error", "coef": np.nan, "se": np.nan, "pvalue": np.nan})

pd.DataFrame(slx_rows).to_csv(out_path("tables", "slx_results_long.csv"), index=False)
pd.DataFrame(slx_moran_rows).to_csv(out_path("tables", "slx_post_moran.csv"), index=False)
print("SLX done. Coefficients count:", len(slx_rows))

SLX done. Coefficients count: 144


## M6 — SDM (Spatial Durbin) with IV-style exposure: first stage A_hat, then spatial lag model

In [ ]:
# M6.1 First stage: A_tilde ~ Z_tilde + X_tilde (by kab), get A_hat
# M6.2 Build WA_hat, then GM_Lag(y_tilde, [A_hat, WA_hat, X_tilde], W)
def first_stage_A_hat(run_df, Z_cols, X_cols):
    run = run_df.copy()
    to_demean = ["A"] + Z_cols + [c for c in X_cols if c in run.columns]
    run = demean_by(run, FE_KEY, to_demean)
    # Only numeric columns for statsmodels (avoids object dtype)
    exog = [c for c in (Z_cols + [c for c in X_cols if c in run.columns and run[c].notna().all()]) if pd.api.types.is_numeric_dtype(run[c])]
    X = np.asarray(sm.add_constant(run[exog]), dtype=np.float64)
    y_a = np.asarray(run["A"], dtype=np.float64)
    groups = run[FE_KEY].astype(str) if run[FE_KEY].dtype.name == "category" else run[FE_KEY]
    mod = sm.OLS(y_a, X).fit(cov_type="cluster", cov_kwds={"groups": groups})
    return mod.predict(X), mod

def run_sdm_iv(run_df, W, Z_cols, X_cols):
    run = run_df.copy()
    A_hat, fs_mod = first_stage_A_hat(run, Z_cols, X_cols)
    run["A_hat"] = A_hat
    run["WA_hat"] = lag_spatial(W, A_hat)
    to_demean = ["y", "A_hat", "WA_hat"] + [c for c in X_cols if c in run.columns]
    run = demean_by(run, FE_KEY, to_demean)
    exog_cols = ["A_hat", "WA_hat"] + [c for c in X_cols if c in run.columns and run[c].notna().all() and pd.api.types.is_numeric_dtype(run[c])]
    X = np.asarray(run[exog_cols], dtype=np.float64)
    y = np.asarray(run["y"], dtype=np.float64)
    # spreg.GM_Lag: y = rho*W*y + X*beta
    try:
        model = spreg.GM_Lag(y, X, w=W, name_y="y", name_x=exog_cols)
        rho = model.rho
        # model.betas layout: [CONSTANT, exog_cols..., rho] — skip constant (idx 0)
        betas = {"rho": float(np.asarray(rho).flat[0])}
        for idx, name in enumerate(exog_cols):
            betas[name] = float(np.asarray(model.betas[idx + 1]).flat[0])
        return model, betas, run
    except Exception as e:
        return None, None, run

# Impacts helper: direct/indirect/total (simplified decomposition)
def compute_impacts(rho, W, beta_A, beta_WA):
    rho = float(np.asarray(rho).flat[0])
    beta_A = float(np.asarray(beta_A).flat[0])
    beta_WA = float(np.asarray(beta_WA).flat[0])
    # Simplified direct/indirect — avoids costly full matrix inverse
    return {"direct": beta_A, "indirect": beta_WA, "total": beta_A + beta_WA}

sdm_rows = []
fs_rows = []
impact_rows = []
for sample in SAMPLES:
    for exposure in EXPOSURES:
        for outcome in OUTCOMES:
            run, _ = build_run_df(sample, exposure, outcome)
            if run is None or len(run) < 20:
                continue
            W = build_W_for_run(run)
            if W is None:
                continue
            A_hat, fs_mod = first_stage_A_hat(run, INSTRUMENTS, X_COLS)
            fs_rows.append({
                "sample": sample, "exposure": exposure, "outcome": outcome,
                "N": int(fs_mod.nobs), "rsquared": getattr(fs_mod, "rsquared", np.nan),
            })
            model, betas, _ = run_sdm_iv(run, W, INSTRUMENTS, X_COLS)
            if model is not None and betas is not None:
                for k, v in betas.items():
                    sdm_rows.append({"sample": sample, "exposure": exposure, "outcome": outcome, "term": k, "coef": v})
                if "A_hat" in betas and "WA_hat" in betas:
                    imp = compute_impacts(model.rho, W, betas["A_hat"], betas["WA_hat"])
                    for k, v in imp.items():
                        impact_rows.append({"sample": sample, "exposure": exposure, "outcome": outcome, "impact": k, "value": v})

pd.DataFrame(sdm_rows).to_csv(out_path("tables", "sdm_results_long.csv"), index=False)
pd.DataFrame(fs_rows).to_csv(out_path("tables", "first_stage_diagnostics.csv"), index=False)
pd.DataFrame(impact_rows).to_csv(out_path("tables", "impacts_sdm.csv"), index=False)
print("SDM done. Coefficients:", len(sdm_rows))

/Users/athamawardi/miniconda3/envs/playgroundenv/lib/python3.14/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:606: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
/Users/athamawardi/miniconda3/envs/playgroundenv/lib/python3.14/site-packages/scipy/sparse/linalg/_matfuncs.py:76: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  Ainv = spsolve(A, I)
/Users/athamawardi/miniconda3/envs/playgroundenv/lib/python3.14/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:606: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
/Users/athamawardi/miniconda3/envs/playgroundenv/lib/python3.14/site-packages/scipy/sparse/linalg/_matfuncs.py:76: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  Ainv = spsolve(A, I)
/Users/athamawardi/miniconda3/envs/playgroundenv/lib/python3.14/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:

SDM done. Coefficients: 156


## I.1 Pemetaan Dampak Kesejahteraan

Subsection:
- **I.1.1** Jangkauan dan Kedekatan Dampak secara Geografis — peta eksposur/jarak dan outcome per desa.
- **I.1.2** Identifikasi Pola Pengelompokan Kesejahteraan — peta LISA / pengelompokan spasial outcome.

## Impact from all SPBUN locations (lokasi_spbu-n.xlsx)

Measure and plot impact using **all** coordinates in `Survey_Map/lokasi_spbu-n.xlsx`. Multiple maps for different angles: impact by exposure/outcome, SPBUN distribution, most-affected villages, and exposure intensity.

In [ ]:
# Load all SPBUN coordinates from lokasi_spbu-n.xlsx (sheet Data, column Koordinat)
PATH_SPBUN_XLSX = os.path.join(ROOT, "lokasi_spbu-n.xlsx")

def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(np.minimum(a, 1)))
    return R * c

def load_spbun_coordinates(path):
    df = pd.read_excel(path, sheet_name="Data")
    out = []
    for v in df["Koordinat"].dropna():
        s = str(v).strip()
        parts = s.replace(",", " ").split()
        if len(parts) >= 2:
            try:
                a, b = float(parts[0]), float(parts[1])
                if -15 <= a <= 10 and 90 <= b <= 150:
                    out.append((a, b))
                elif -15 <= b <= 10 and 90 <= a <= 150:
                    out.append((b, a))
            except Exception:
                pass
    return out

spbun_coords = load_spbun_coordinates(PATH_SPBUN_XLSX) if os.path.exists(PATH_SPBUN_XLSX) else []
print("Loaded SPBUN coordinates:", len(spbun_coords))

def run_simulation_map_multi(sample, outcome, exposure, spbun_lat_lon_list):
    """Impact from all SPBUN locations (cumulative dA). Same controls (X_COLS), SDM multiplier."""
    if not spbun_lat_lon_list:
        return None
    run, _ = build_run_df(sample, exposure, outcome)
    if run is None or len(run) < 20:
        return None
    W = build_W_for_run(run)
    if W is None:
        return None
    lat = run["lat_v"].values
    lon = run["lon_v"].values
    n = len(run)
    # Cumulative dA from all SPBUNs (linear approximation)
    dA = np.zeros(n, dtype=np.float64)
    for (slat, slon) in spbun_lat_lon_list:
        D = np.array([haversine_km(lat[i], lon[i], slat, slon) for i in range(n)])
        if exposure == "A_v_log":
            dA += (-np.log1p(D))  # sum over sources
        elif exposure == "Ker5_w1":
            dA += (D <= 5).astype(np.float64)
        elif exposure == "Ker10_w1":
            dA += (D <= 10).astype(np.float64)
    if exposure == "A_v_log":
        dA = dA - run["A"].values  # dA = A_new - A_current; A_new = sum we just computed
    model, betas, _ = run_sdm_iv(run, W, INSTRUMENTS, X_COLS)
    if model is None or betas is None:
        return None
    rho = float(np.asarray(betas.get("rho", 0)).flat[0])
    beta_A = float(np.asarray(betas.get("A_hat", 0)).flat[0])
    theta = float(np.asarray(betas.get("WA_hat", 0)).flat[0])
    WdA = lag_spatial(W, dA)
    rhs = beta_A * dA + theta * WdA
    # Solve (I - rho*W) @ dy = rhs directly (avoids kernel-crashing dense inverse)
    A_mat = (sparse.eye(W.n) - rho * W.sparse).tocsc()
    dy = sparse.linalg.spsolve(A_mat, rhs)
    run = run.copy()
    run["dy"] = dy
    run["dA"] = dA
    return run

# ---- Map 1: SPBUN distribution only (all coordinates on basemap) ----
if spbun_coords and BASEMAP is not None:
    fig, ax = plt.subplots(figsize=(14, 8))
    BASEMAP.plot(ax=ax, facecolor="lightgrey", edgecolor="darkgrey", linewidth=0.5, zorder=0)
    slat = [c[0] for c in spbun_coords]
    slon = [c[1] for c in spbun_coords]
    ax.scatter(slon, slat, s=4, c="darkred", alpha=0.6, label=f"SPBUN (n={len(spbun_coords)})")
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_title("Lokasi SPBUN (lokasi_spbu-n.xlsx) — distribusi geografis")
    ax.legend()
    ax.set_aspect("equal")
    plt.tight_layout()
    plt.savefig(out_path("maps", "spbun_lokasi_distribution.png"), dpi=150)
    plt.close()
    print("Saved spbun_lokasi_distribution.png")

# ---- Maps 2–4: Impact (Δy) from all file SPBUNs by exposure, with SPBUN overlay — Pesisir & full sample ----
for sample in SAMPLES:
    slab = "Pesisir" if sample == "COAST" else "Seluruh desa (incl. non-pesisir)"
    for exposure in EXPOSURES:
        for outcome in OUTCOMES:
            sim = run_simulation_map_multi(sample, outcome, exposure, spbun_coords)
            if sim is None:
                print(f"Skip multi {sample} {exposure} {outcome}")
                continue
            fig, ax = plt.subplots(figsize=(20, 8))
            if BASEMAP is not None:
                BASEMAP.plot(ax=ax, facecolor="lightgrey", edgecolor="darkgrey", linewidth=0.5, zorder=0)
            sc = ax.scatter(sim["lon_v"], sim["lat_v"], c=sim["dy"], s=2, cmap="RdBu")
            if spbun_coords:
                ax.scatter([c[1] for c in spbun_coords], [c[0] for c in spbun_coords], s=8, c="black", marker=".", alpha=0.5, label="SPBUN")
            plt.colorbar(sc, ax=ax, label="Δy (counterfactual)")
            ax.set_xlabel("Longitude")
            ax.set_ylabel("Latitude")
            ax.set_title(f"Dampak dari semua lokasi SPBUN — {slab} — {exposure} — {outcome}")
            ax.legend(loc="upper right")
            plt.tight_layout()
            plt.savefig(out_path("maps", f"impact_all_spbun_{exposure}_{outcome}_{sample}.png"), dpi=150)
            plt.close()
            print("Saved", f"impact_all_spbun_{exposure}_{outcome}_{sample}.png")

# # ---- Map 5: Villages most affected (top 10% |Δy|) — Pesisir & full sample, with SPBUN ----
# if spbun_coords:
#     for sample in SAMPLES:
#         slab = "Pesisir" if sample == "COAST" else "Seluruh desa (incl. non-pesisir)"
#         sim = run_simulation_map_multi(sample, "lnpov_z", "A_v_log", spbun_coords)
#         if sim is None:
#             continue
#         q90 = sim["dy"].abs().quantile(0.9)
#         top = sim["dy"].abs() >= q90
#         fig, ax = plt.subplots(figsize=(14, 8))
#         if BASEMAP is not None:
#             BASEMAP.plot(ax=ax, facecolor="lightgrey", edgecolor="darkgrey", linewidth=0.5, zorder=0)
#         ax.scatter(sim.loc[~top, "lon_v"], sim.loc[~top, "lat_v"], s=1, c="lightgray", alpha=0.5)
#         ax.scatter(sim.loc[top, "lon_v"], sim.loc[top, "lat_v"], c=sim.loc[top, "dy"], s=8, cmap="RdBu_r")
#         ax.scatter([c[1] for c in spbun_coords], [c[0] for c in spbun_coords], s=6, c="black", marker=".", alpha=0.6, label="SPBUN")
#         ax.set_xlabel("Longitude")
#         ax.set_ylabel("Latitude")
#         ax.set_title(f"Desa paling terdampak (top 10% |Δy|) — {slab} — lnpov_z, A_v_log")
#         plt.tight_layout()
#         plt.savefig(out_path("maps", f"impact_most_affected_lnpov_A_v_log_{sample}.png"), dpi=150)
#         plt.close()
#         print("Saved", f"impact_most_affected_lnpov_A_v_log_{sample}.png")

# # ---- Map 6: Exposure intensity 10 km — Pesisir & full sample ----
# if spbun_coords:
#     for sample in SAMPLES:
#         slab = "Pesisir" if sample == "COAST" else "Seluruh desa (incl. non-pesisir)"
#         run, _ = build_run_df(sample, "A_v_log", "lnpov_z")
#         if run is None or len(run) < 20:
#             continue
#         lat, lon = run["lat_v"].values, run["lon_v"].values
#         count_10km = np.zeros(len(run))
#         for (slat, slon) in spbun_coords:
#             D = np.array([haversine_km(lat[i], lon[i], slat, slon) for i in range(len(run))])
#             count_10km += (D <= 10).astype(np.float64)
#         run = run.copy()
#         run["n_spbun_10km"] = count_10km
#         fig, ax = plt.subplots(figsize=(14, 8))
#         if BASEMAP is not None:
#             BASEMAP.plot(ax=ax, facecolor="lightgrey", edgecolor="darkgrey", linewidth=0.5, zorder=0)
#         sc = ax.scatter(run["lon_v"], run["lat_v"], c=run["n_spbun_10km"], s=2, cmap="YlOrRd")
#         ax.scatter([c[1] for c in spbun_coords], [c[0] for c in spbun_coords], s=6, c="black", marker=".", alpha=0.5, label="SPBUN")
#         plt.colorbar(sc, ax=ax, label="Jumlah SPBUN dalam 10 km")
#         ax.set_xlabel("Longitude")
#         ax.set_ylabel("Latitude")
#         ax.set_title(f"Intensitas kedekatan (10 km) — {slab}")
#         plt.tight_layout()
#         plt.savefig(out_path("maps", f"exposure_intensity_spbun_10km_{sample}.png"), dpi=150)
#         plt.close()
#         print("Saved", f"exposure_intensity_spbun_10km_{sample}.png")

# # ---- Map 6b: Exposure intensity 5 km — Pesisir & full sample ----
# if spbun_coords:
#     for sample in SAMPLES:
#         slab = "Pesisir" if sample == "COAST" else "Seluruh desa (incl. non-pesisir)"
#         run, _ = build_run_df(sample, "A_v_log", "lnpov_z")
#         if run is None or len(run) < 20:
#             continue
#         lat, lon = run["lat_v"].values, run["lon_v"].values
#         count_5km = np.zeros(len(run))
#         for (slat, slon) in spbun_coords:
#             D = np.array([haversine_km(lat[i], lon[i], slat, slon) for i in range(len(run))])
#             count_5km += (D <= 5).astype(np.float64)
#         run = run.copy()
#         run["n_spbun_5km"] = count_5km
#         fig, ax = plt.subplots(figsize=(14, 8))
#         if BASEMAP is not None:
#             BASEMAP.plot(ax=ax, facecolor="lightgrey", edgecolor="darkgrey", linewidth=0.5, zorder=0)
#         sc = ax.scatter(run["lon_v"], run["lat_v"], c=run["n_spbun_5km"], s=2, cmap="YlOrRd")
#         ax.scatter([c[1] for c in spbun_coords], [c[0] for c in spbun_coords], s=6, c="black", marker=".", alpha=0.5, label="SPBUN")
#         plt.colorbar(sc, ax=ax, label="Jumlah SPBUN dalam 5 km")
#         ax.set_xlabel("Longitude")
#         ax.set_ylabel("Latitude")
#         ax.set_title(f"Intensitas kedekatan (5 km) — {slab}")
#         plt.tight_layout()
#         plt.savefig(out_path("maps", f"exposure_intensity_spbun_5km_{sample}.png"), dpi=150)
#         plt.close()
#         print("Saved", f"exposure_intensity_spbun_5km_{sample}.png")

# # ---- Map 7: Two-panel lnpov vs Wbasic impact — Pesisir & full sample, SPBUN overlay ----
# if spbun_coords:
#     for sample in SAMPLES:
#         slab = "Pesisir" if sample == "COAST" else "Seluruh desa (incl. non-pesisir)"
#         fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
#         for ax, outcome in [(ax1, "lnpov_z"), (ax2, "Wbasic_z")]:
#             sim = run_simulation_map_multi(sample, outcome, "A_v_log", spbun_coords)
#             if sim is None:
#                 continue
#             if BASEMAP is not None:
#                 BASEMAP.plot(ax=ax, facecolor="lightgrey", edgecolor="darkgrey", linewidth=0.5, zorder=0)
#             ax.scatter(sim["lon_v"], sim["lat_v"], c=sim["dy"], s=2, cmap="RdBu_r")
#             ax.scatter([c[1] for c in spbun_coords], [c[0] for c in spbun_coords], s=6, c="black", marker=".", alpha=0.5)
#             ax.set_title(f"Δy — {outcome} (A_v_log)")
#             ax.set_xlabel("Longitude")
#             ax.set_ylabel("Latitude")
#         plt.suptitle(f"Perbandingan dampak — {slab}: lnpov_z vs Wbasic_z")
#         plt.tight_layout()
#         plt.savefig(out_path("maps", f"impact_compare_lnpov_vs_Wbasic_{sample}.png"), dpi=150)
#         plt.close()
#         print("Saved", f"impact_compare_lnpov_vs_Wbasic_{sample}.png")

# # ---- Map 8: LISA of impact (Δy) — Pesisir & full sample, SPBUN overlay ----
# if spbun_coords:
#     for sample in SAMPLES:
#         slab = "Pesisir" if sample == "COAST" else "Seluruh desa (incl. non-pesisir)"
#         for outcome in OUTCOMES:
#             sim = run_simulation_map_multi(sample, outcome, "A_v_log", spbun_coords)
#             if sim is None or len(sim) < 100:
#                 continue
#             W_sim = build_W_for_run(sim)
#             if W_sim is None:
#                 continue
#             try:
#                 lisa = Moran_Local(sim["dy"].values.astype(np.float64), W_sim, permutations=199)
#                 sig = lisa.p_sim < 0.05
#                 q = np.full(len(sim), np.nan)
#                 q[sig] = lisa.q[sig]
#                 fig, ax = plt.subplots(figsize=(14, 8))
#                 if BASEMAP is not None:
#                     BASEMAP.plot(ax=ax, facecolor="lightgrey", edgecolor="darkgrey", linewidth=0.5, zorder=0)
#                 sc = ax.scatter(sim["lon_v"], sim["lat_v"], c=q, s=3, cmap="RdYlBu_r", vmin=1, vmax=4)
#                 ax.scatter([c[1] for c in spbun_coords], [c[0] for c in spbun_coords], s=6, c="black", marker=".", alpha=0.5, label="SPBUN")
#                 cbar = plt.colorbar(sc, ax=ax)
#                 cbar.set_ticks([1, 2, 3, 4])
#                 cbar.set_ticklabels(["HH", "LH", "LL", "HL"])
#                 cbar.set_label("LISA cluster (p<0.05)")
#                 ax.set_xlabel("Longitude")
#                 ax.set_ylabel("Latitude")
#                 ax.set_title(f"LISA dampak (Δy) — {slab} — {outcome}")
#                 ax.legend(loc="upper right")
#                 plt.tight_layout()
#                 plt.savefig(out_path("maps", f"lisa_impact_dy_{outcome}_{sample}.png"), dpi=150)
#                 plt.close()
#                 print("Saved", f"lisa_impact_dy_{outcome}_{sample}.png")
#             except Exception as e:
#                 print(f"LISA impact skip {sample} {outcome}:", e)

# # ---- Map 9: LISA of exposure intensity (5 km) — Pesisir & full sample ----
# if spbun_coords:
#     for sample in SAMPLES:
#         slab = "Pesisir" if sample == "COAST" else "Seluruh desa (incl. non-pesisir)"
#         run, _ = build_run_df(sample, "A_v_log", "lnpov_z")
#         if run is None or len(run) < 100:
#             continue
#         lat, lon = run["lat_v"].values, run["lon_v"].values
#         count_5km = np.zeros(len(run))
#         for (slat, slon) in spbun_coords:
#             D = np.array([haversine_km(lat[i], lon[i], slat, slon) for i in range(len(run))])
#             count_5km += (D <= 5).astype(np.float64)
#         run = run.copy()
#         run["n_spbun_5km"] = count_5km
#         W_run = build_W_for_run(run)
#         if W_run is not None:
#             try:
#                 lisa5 = Moran_Local(run["n_spbun_5km"].values.astype(np.float64), W_run, permutations=199)
#                 sig5 = lisa5.p_sim < 0.05
#                 q5 = np.full(len(run), np.nan)
#                 q5[sig5] = lisa5.q[sig5]
#                 fig, ax = plt.subplots(figsize=(14, 8))
#                 if BASEMAP is not None:
#                     BASEMAP.plot(ax=ax, facecolor="lightgrey", edgecolor="darkgrey", linewidth=0.5, zorder=0)
#                 sc = ax.scatter(run["lon_v"], run["lat_v"], c=q5, s=3, cmap="RdYlBu_r", vmin=1, vmax=4)
#                 ax.scatter([c[1] for c in spbun_coords], [c[0] for c in spbun_coords], s=6, c="black", marker=".", alpha=0.5, label="SPBUN")
#                 cbar = plt.colorbar(sc, ax=ax)
#                 cbar.set_ticks([1, 2, 3, 4])
#                 cbar.set_ticklabels(["HH", "LH", "LL", "HL"])
#                 cbar.set_label("LISA cluster (p<0.05)")
#                 ax.set_xlabel("Longitude")
#                 ax.set_ylabel("Latitude")
#                 ax.set_title(f"LISA intensitas 5 km — {slab}")
#                 ax.legend(loc="upper right")
#                 plt.tight_layout()
#                 plt.savefig(out_path("maps", f"lisa_intensity_5km_{sample}.png"), dpi=150)
#                 plt.close()
#                 print("Saved", f"lisa_intensity_5km_{sample}.png")
#             except Exception as e:
#                 print("LISA 5km skip", sample, e)

# # ---- Map 10: LISA of exposure intensity (10 km) — Pesisir & full sample ----
# if spbun_coords:
#     for sample in SAMPLES:
#         slab = "Pesisir" if sample == "COAST" else "Seluruh desa (incl. non-pesisir)"
#         run, _ = build_run_df(sample, "A_v_log", "lnpov_z")
#         if run is None or len(run) < 100:
#             continue
#         lat, lon = run["lat_v"].values, run["lon_v"].values
#         count_10km = np.zeros(len(run))
#         for (slat, slon) in spbun_coords:
#             D = np.array([haversine_km(lat[i], lon[i], slat, slon) for i in range(len(run))])
#             count_10km += (D <= 10).astype(np.float64)
#         run = run.copy()
#         run["n_spbun_10km"] = count_10km
#         W_run = build_W_for_run(run)
#         if W_run is not None:
#             try:
#                 lisa10 = Moran_Local(run["n_spbun_10km"].values.astype(np.float64), W_run, permutations=199)
#                 sig10 = lisa10.p_sim < 0.05
#                 q10 = np.full(len(run), np.nan)
#                 q10[sig10] = lisa10.q[sig10]
#                 fig, ax = plt.subplots(figsize=(14, 8))
#                 if BASEMAP is not None:
#                     BASEMAP.plot(ax=ax, facecolor="lightgrey", edgecolor="darkgrey", linewidth=0.5, zorder=0)
#                 sc = ax.scatter(run["lon_v"], run["lat_v"], c=q10, s=3, cmap="RdYlBu_r", vmin=1, vmax=4)
#                 ax.scatter([c[1] for c in spbun_coords], [c[0] for c in spbun_coords], s=6, c="black", marker=".", alpha=0.5, label="SPBUN")
#                 cbar = plt.colorbar(sc, ax=ax)
#                 cbar.set_ticks([1, 2, 3, 4])
#                 cbar.set_ticklabels(["HH", "LH", "LL", "HL"])
#                 cbar.set_label("LISA cluster (p<0.05)")
#                 ax.set_xlabel("Longitude")
#                 ax.set_ylabel("Latitude")
#                 ax.set_title(f"LISA intensitas 10 km — {slab}")
#                 ax.legend(loc="upper right")
#                 plt.tight_layout()
#                 plt.savefig(out_path("maps", f"lisa_intensity_10km_{sample}.png"), dpi=150)
#                 plt.close()
#                 print("Saved", f"lisa_intensity_10km_{sample}.png")
#             except Exception as e:
#                 print("LISA 10km skip", sample, e)

# # ---- Map 11: Two-panel LISA 5 km vs 10 km — Pesisir & full sample ----
# if spbun_coords:
#     for sample in SAMPLES:
#         slab = "Pesisir" if sample == "COAST" else "Seluruh desa (incl. non-pesisir)"
#         run, _ = build_run_df(sample, "A_v_log", "lnpov_z")
#         if run is None or len(run) < 100:
#             continue
#         lat, lon = run["lat_v"].values, run["lon_v"].values
#         count_5km = np.zeros(len(run))
#         count_10km = np.zeros(len(run))
#         for (slat, slon) in spbun_coords:
#             D = np.array([haversine_km(lat[i], lon[i], slat, slon) for i in range(len(run))])
#             count_5km += (D <= 5).astype(np.float64)
#             count_10km += (D <= 10).astype(np.float64)
#         run = run.copy()
#         run["n_spbun_5km"], run["n_spbun_10km"] = count_5km, count_10km
#         W_run = build_W_for_run(run)
#         if W_run is not None:
#             try:
#                 lisa5 = Moran_Local(run["n_spbun_5km"].values.astype(np.float64), W_run, permutations=199)
#                 lisa10 = Moran_Local(run["n_spbun_10km"].values.astype(np.float64), W_run, permutations=199)
#                 q5 = np.where(lisa5.p_sim < 0.05, lisa5.q, np.nan)
#                 q10 = np.where(lisa10.p_sim < 0.05, lisa10.q, np.nan)
#                 fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
#                 for ax, q, title in [(ax1, q5, "5 km"), (ax2, q10, "10 km")]:
#                     if BASEMAP is not None:
#                         BASEMAP.plot(ax=ax, facecolor="lightgrey", edgecolor="darkgrey", linewidth=0.5, zorder=0)
#                     sc = ax.scatter(run["lon_v"], run["lat_v"], c=q, s=3, cmap="RdYlBu_r", vmin=1, vmax=4)
#                     ax.scatter([c[1] for c in spbun_coords], [c[0] for c in spbun_coords], s=6, c="black", marker=".", alpha=0.5)
#                     ax.set_title(f"LISA intensitas — {title}")
#                     ax.set_xlabel("Longitude")
#                     ax.set_ylabel("Latitude")
#                 plt.suptitle(f"Perbandingan LISA intensitas — {slab}: 5 km vs 10 km")
#                 plt.tight_layout()
#                 plt.savefig(out_path("maps", f"lisa_intensity_5km_vs_10km_{sample}.png"), dpi=150)
#                 plt.close()
#                 print("Saved", f"lisa_intensity_5km_vs_10km_{sample}.png")
#             except Exception as e:
#                 print("LISA 5km vs 10km skip", sample, e)

# print("All SPBUN-coordinate maps done.")

Loaded SPBUN coordinates: 374
Saved spbun_lokasi_distribution.png


/Users/athamawardi/miniconda3/envs/playgroundenv/lib/python3.14/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:606: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
/Users/athamawardi/miniconda3/envs/playgroundenv/lib/python3.14/site-packages/scipy/sparse/linalg/_matfuncs.py:76: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  Ainv = spsolve(A, I)


Saved impact_all_spbun_A_v_log_lnpov_z_COAST.png


/Users/athamawardi/miniconda3/envs/playgroundenv/lib/python3.14/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:606: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
/Users/athamawardi/miniconda3/envs/playgroundenv/lib/python3.14/site-packages/scipy/sparse/linalg/_matfuncs.py:76: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  Ainv = spsolve(A, I)


Saved impact_all_spbun_A_v_log_Wbasic_z_COAST.png


...

Saved lisa_impact_dy_Wbasic_z_COAST.png


/Users/athamawardi/miniconda3/envs/playgroundenv/lib/python3.14/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:606: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
/Users/athamawardi/miniconda3/envs/playgroundenv/lib/python3.14/site-packages/scipy/sparse/linalg/_matfuncs.py:76: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  Ainv = spsolve(A, I)


Saved lisa_impact_dy_lnpov_z_ALL.png


/Users/athamawardi/miniconda3/envs/playgroundenv/lib/python3.14/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:606: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
/Users/athamawardi/miniconda3/envs/playgroundenv/lib/python3.14/site-packages/scipy/sparse/linalg/_matfuncs.py:76: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  Ainv = spsolve(A, I)


Saved lisa_impact_dy_Wbasic_z_ALL.png
Saved lisa_intensity_5km_COAST.png
Saved lisa_intensity_5km_ALL.png
Saved lisa_intensity_10km_COAST.png
Saved lisa_intensity_10km_ALL.png
Saved lisa_intensity_5km_vs_10km_COAST.png
Saved lisa_intensity_5km_vs_10km_ALL.png
All SPBUN-coordinate maps done.


In [14]:
# --- I.1 & I.1.1 & I.1.2: Welfare maps, proximity/scope, clustering (basemap from Survey_Map) ---
def plot_on_basemap(ax, lon, lat, c, title, cmap="viridis", clabel="", s=2):
    if BASEMAP is not None:
        BASEMAP.plot(ax=ax, facecolor="lightgrey", edgecolor="darkgrey", linewidth=0.5, zorder=0)
    sc = ax.scatter(lon, lat, c=c, s=s, cmap=cmap)
    plt.colorbar(sc, ax=ax, label=clabel)
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_title(title)
    ax.set_aspect("equal")

# I.1 — Pemetaan Dampak Kesejahteraan: outcome levels by village
for sample in SAMPLES:
    df = DATASETS[sample]
    for outcome in OUTCOMES:
        if outcome not in df.columns:
            continue
        r = df[["lat_v", "lon_v", outcome]].dropna(how="any")
        if len(r) < 50:
            continue
        fig, ax = plt.subplots(figsize=(14, 8))
        slab = "Pesisir" if sample == "COAST" else "Seluruh desa (incl. non-pesisir)"
        plot_on_basemap(ax, r["lon_v"], r["lat_v"], r[outcome], f"I.1 Kesejahteraan — {slab} — {outcome}", clabel=outcome)
        plt.tight_layout()
        plt.savefig(out_path("maps", f"welfare_{sample}_{outcome}.png"), dpi=150)
        plt.close()
        print("Saved", f"welfare_{sample}_{outcome}.png")

# I.1.1 — Jangkauan dan Kedekatan Dampak: exposure (proximity) by village
for sample in SAMPLES:
    df = DATASETS[sample]
    for exp in ["A_v_log", "Ker5_w1", "Ker10_w1"]:
        if exp not in df.columns:
            continue
        r = df[["lat_v", "lon_v", exp]].dropna(how="any")
        if len(r) < 50:
            continue
        fig, ax = plt.subplots(figsize=(14, 8))
        slab = "Pesisir" if sample == "COAST" else "Seluruh desa (incl. non-pesisir)"
        plot_on_basemap(ax, r["lon_v"], r["lat_v"], r[exp], f"I.1.1 Kedekatan — {slab} — {exp}", clabel=exp)
        plt.tight_layout()
        plt.savefig(out_path("maps", f"proximity_{sample}_{exp}.png"), dpi=150)
        plt.close()
        print("Saved", f"proximity_{sample}_{exp}.png")

# I.1.2 — Pola Pengelompokan Kesejahteraan: LISA (local Moran) clusters
for sample in SAMPLES:
    df = DATASETS[sample]
    r = df[["lat_v", "lon_v"] + [c for c in OUTCOMES if c in df.columns]].dropna(how="any")
    if len(r) < 100:
        continue
    W = build_W_for_run(r)
    if W is None:
        continue
    for outcome in OUTCOMES:
        if outcome not in r.columns:
            continue
        y = r[outcome].values.astype(np.float64)
        try:
            lisa = Moran_Local(y, W, permutations=199)
            # 1=HH, 2=LH, 3=LL, 4=HL; use only significant (e.g. p<0.05)
            sig = lisa.p_sim < 0.05
            q = np.full(len(y), np.nan)
            q[sig] = lisa.q[sig]
            fig, ax = plt.subplots(figsize=(14, 8))
            if BASEMAP is not None:
                BASEMAP.plot(ax=ax, facecolor="lightgrey", edgecolor="darkgrey", linewidth=0.5, zorder=0)
            sc = ax.scatter(r["lon_v"], r["lat_v"], c=q, s=3, cmap="RdYlBu_r", vmin=1, vmax=4)
            cbar = plt.colorbar(sc, ax=ax, label="LISA cluster (1=HH,2=LH,3=LL,4=HL)")
            cbar.set_ticks([1, 2, 3, 4])
            cbar.set_ticklabels(["HH", "LH", "LL", "HL"])
            ax.set_xlabel("Longitude")
            ax.set_ylabel("Latitude")
            slab = "Pesisir" if sample == "COAST" else "Seluruh desa (incl. non-pesisir)"
            ax.set_title(f"I.1.2 Pengelompokan Kesejahteraan — {slab} — {outcome} (LISA, p<0.05)")
            ax.set_aspect("equal")
            plt.tight_layout()
            plt.savefig(out_path("maps", f"lisa_{sample}_{outcome}.png"), dpi=150)
            plt.close()
            print("Saved", f"lisa_{sample}_{outcome}.png")
        except Exception as e:
            print(f"LISA skip {sample} {outcome}:", e)

# --- Comparison: Pesisir vs Seluruh desa (side-by-side) to see impact on neighbouring non-coastal villages ---
for outcome in OUTCOMES:
    data = {}
    for sample in SAMPLES:
        df = DATASETS[sample]
        if outcome not in df.columns:
            continue
        r = df[["lat_v", "lon_v", outcome]].dropna(how="any")
        if len(r) < 50:
            continue
        data[sample] = r
    if "COAST" not in data or "ALL" not in data:
        continue
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
    plot_on_basemap(ax1, data["COAST"]["lon_v"], data["COAST"]["lat_v"], data["COAST"][outcome], f"Pesisir — {outcome}", clabel=outcome, s=3)
    plot_on_basemap(ax2, data["ALL"]["lon_v"], data["ALL"]["lat_v"], data["ALL"][outcome], f"Seluruh desa (incl. non-pesisir) — {outcome}", clabel=outcome, s=1)
    plt.suptitle(f"I.1 Perbandingan: Pesisir vs Seluruh desa — {outcome}", fontsize=14)
    plt.tight_layout()
    plt.savefig(out_path("maps", f"compare_COAST_vs_ALL_{outcome}.png"), dpi=150)
    plt.close()
    print("Saved", f"compare_COAST_vs_ALL_{outcome}.png")

print("I.1 / I.1.1 / I.1.2 maps done (Pesisir + full sample + comparison).")

Saved welfare_COAST_lnpov_z.png
Saved welfare_COAST_Wbasic_z.png
Saved welfare_ALL_lnpov_z.png
Saved welfare_ALL_Wbasic_z.png
Saved proximity_COAST_A_v_log.png
Saved proximity_COAST_Ker5_w1.png
Saved proximity_COAST_Ker10_w1.png
Saved proximity_ALL_A_v_log.png
Saved proximity_ALL_Ker5_w1.png
Saved proximity_ALL_Ker10_w1.png
Saved lisa_COAST_lnpov_z.png
Saved lisa_COAST_Wbasic_z.png
Saved lisa_ALL_lnpov_z.png
Saved lisa_ALL_Wbasic_z.png
Saved compare_COAST_vs_ALL_lnpov_z.png
Saved compare_COAST_vs_ALL_Wbasic_z.png
I.1 / I.1.1 / I.1.2 maps done (Pesisir + full sample + comparison).


## M7 — SEM with ring-threshold terms (A0_5, A5_10, A10_15); spatial error

In [18]:
# M7: Ring exposures from C5/C10/C15 or Ker5/Ker10/Ker15; then GM_Error
def add_ring_exposure(df):
    d = df.copy()
    if "C5_w1" in d.columns and "C10_w1" in d.columns and "C15_w1" in d.columns:
        d["A0_5"] = d["C5_w1"]
        d["A5_10"] = (d["C10_w1"] - d["C5_w1"]).clip(lower=0)
        d["A10_15"] = (d["C15_w1"] - d["C10_w1"]).clip(lower=0)
    elif "Ker5_w1" in d.columns and "Ker10_w1" in d.columns:
        d["A0_5"] = d["Ker5_w1"]
        d["A5_10"] = (d["Ker10_w1"] - d["Ker5_w1"]).clip(lower=0)
        d["A10_15"] = (d["Ker15_w1"] - d["Ker10_w1"]).clip(lower=0) if "Ker15_w1" in d.columns else 0.0
    else:
        d["A0_5"] = np.nan
        d["A5_10"] = np.nan
        d["A10_15"] = np.nan
    return d

RING_COLS = ["A0_5", "A5_10", "A10_15"]

sem_rows = []
sem_lambda_rows = []
for sample in SAMPLES:
    run = DATASETS[sample].copy()
    run = add_ring_exposure(run)
    for outcome in OUTCOMES:
        if outcome not in run.columns:
            continue
        run["y"] = run[outcome]
        req = ["iddesa", FE_KEY, "lat_v", "lon_v", "y"] + RING_COLS + X_COLS
        req = [c for c in req if c in run.columns]
        r = run[req].dropna(how="any")
        if len(r) < 20:
            continue
        W = build_W_for_run(r)
        if W is None:
            continue
        r = demean_by(r, FE_KEY, ["y"] + RING_COLS + [c for c in X_COLS if c in r.columns])
        exog = [c for c in RING_COLS + X_COLS if c in r.columns and r[c].notna().all() and pd.api.types.is_numeric_dtype(r[c])]
        if len(exog) < 2:
            continue
        X = np.asarray(r[exog], dtype=np.float64)
        y = np.asarray(r["y"], dtype=np.float64)
        try:
            model = spreg.GM_Error(y, X, w=W, name_x=exog)
            # GM_Error has no .lam; spatial coefficient lambda is last in model.betas
            lam = float(np.asarray(model.betas).flat[-1])
            nk = len(model.betas.flat)
            lambda_se = float(model.std_err.flat[-1]) if len(model.std_err.flat) >= nk else np.nan
            sem_lambda_rows.append({"sample": sample, "outcome": outcome, "lambda": lam, "lambda_se": lambda_se})
            for i, name in enumerate(exog):
                sem_rows.append({"sample": sample, "outcome": outcome, "term": name, "coef": model.betas[i], "se": np.sqrt(model.vm[i, i])})
        except Exception as e:
            print("SEM skip", sample, outcome, e)

pd.DataFrame(sem_rows).to_csv(out_path("tables", "sem_ring_results_long.csv"), index=False)
pd.DataFrame(sem_lambda_rows).to_csv(out_path("tables", "sem_lambda_table.csv"), index=False)
print("SEM ring done. Rows:", len(sem_rows))

SEM ring done. Rows: 52


## Simulation map: new SPBUN → Δy (A_v_log, Ker5_w1, Ker10_w1) — full sample (ALL) with controls

In [ ]:
# Counterfactual: one hypothetical SPBUN location; compute ΔA = A_new - A_current; Δy from SDM multiplier
# Supports A_v_log, Ker5_w1, Ker10_w1. Uses full sample (ALL) with controls (X_COLS).
# Note: haversine_km is already defined above (SPBUN multi-map cell)

def run_simulation_map(sample, outcome, exposure, new_spbun_lat, new_spbun_lon):
    run, _ = build_run_df(sample, exposure, outcome)
    if run is None or len(run) < 20:
        return None
    W = build_W_for_run(run)
    if W is None:
        return None
    lat = run["lat_v"].values
    lon = run["lon_v"].values
    D_new = np.array([haversine_km(lat[i], lon[i], new_spbun_lat, new_spbun_lon) for i in range(len(run))])
    A_cur = run["A"].values
    if exposure == "A_v_log":
        A_new = -np.log1p(D_new)
        dA = A_new - A_cur
    elif exposure == "Ker5_w1":
        dA = (D_new <= 5).astype(np.float64)  # +1 SPBUN within 5 km
    elif exposure == "Ker10_w1":
        dA = (D_new <= 10).astype(np.float64)  # +1 SPBUN within 10 km
    else:
        return None
    # SDM: Δy = (I - ρW)^{-1} (β*dA + θ*W*dA). Same controls (X_COLS) as main SDM.
    model, betas, _ = run_sdm_iv(run, W, INSTRUMENTS, X_COLS)
    if model is None or betas is None:
        return None
    rho = float(np.asarray(betas.get("rho", 0)).flat[0])
    beta_A = float(np.asarray(betas.get("A_hat", 0)).flat[0])
    theta = float(np.asarray(betas.get("WA_hat", 0)).flat[0])
    WdA = lag_spatial(W, dA)
    rhs = beta_A * dA + theta * WdA
    # Solve (I - rho*W) @ dy = rhs directly (avoids kernel-crashing dense inverse)
    A_mat = (sparse.eye(W.n) - rho * W.sparse).tocsc()
    dy = sparse.linalg.spsolve(A_mat, rhs)
    run = run.copy()
    run["dy"] = dy
    return run

# Hypothetical SPBUN at -6.2, 106.8 (Java). Plot for Pesisir (COAST) and full sample (ALL) with controls; all three exposures.
NEW_SPBUN_LAT, NEW_SPBUN_LON = -6.2, 106.8
for sample in SAMPLES:
    slab = "Pesisir" if sample == "COAST" else "Seluruh desa (incl. non-pesisir)"
    for exposure in EXPOSURES:
        for outcome in OUTCOMES:
            sim = run_simulation_map(sample, outcome, exposure, NEW_SPBUN_LAT, NEW_SPBUN_LON)
            if sim is None:
                print(f"Skip sim {sample} {exposure} {outcome}")
                continue
            fig, ax = plt.subplots(figsize=(20, 8))
            if BASEMAP is not None:
                BASEMAP.plot(ax=ax, facecolor="lightgrey", edgecolor="darkgrey", linewidth=0.5, zorder=0)
            sc = ax.scatter(sim["lon_v"], sim["lat_v"], c=sim["dy"], s=1, cmap="RdBu_r")
            plt.colorbar(sc, ax=ax, label="Δy (counterfactual)")
            ax.set_xlabel("Longitude")
            ax.set_ylabel("Latitude")
            ax.set_title(f"Δy — {slab} — {exposure} — {outcome} (controls: X_COLS)")
            plt.tight_layout()
            fname = f"sim_dy_{exposure}_{sample}_{outcome}.png"
            plt.savefig(out_path("maps", fname), dpi=150)
            plt.close()
            print("Saved", fname)

/Users/athamawardi/miniconda3/envs/playgroundenv/lib/python3.14/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:606: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
/Users/athamawardi/miniconda3/envs/playgroundenv/lib/python3.14/site-packages/scipy/sparse/linalg/_matfuncs.py:76: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  Ainv = spsolve(A, I)


Saved sim_dy_A_v_log_COAST_lnpov_z.png


/Users/athamawardi/miniconda3/envs/playgroundenv/lib/python3.14/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:606: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
/Users/athamawardi/miniconda3/envs/playgroundenv/lib/python3.14/site-packages/scipy/sparse/linalg/_matfuncs.py:76: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  Ainv = spsolve(A, I)


Saved sim_dy_A_v_log_COAST_Wbasic_z.png


/Users/athamawardi/miniconda3/envs/playgroundenv/lib/python3.14/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:606: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
/Users/athamawardi/miniconda3/envs/playgroundenv/lib/python3.14/site-packages/scipy/sparse/linalg/_matfuncs.py:76: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  Ainv = spsolve(A, I)


...

Saved sim_dy_Ker10_w1_COAST_lnpov_z.png


/Users/athamawardi/miniconda3/envs/playgroundenv/lib/python3.14/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:606: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
/Users/athamawardi/miniconda3/envs/playgroundenv/lib/python3.14/site-packages/scipy/sparse/linalg/_matfuncs.py:76: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  Ainv = spsolve(A, I)


Saved sim_dy_Ker10_w1_COAST_Wbasic_z.png


/Users/athamawardi/miniconda3/envs/playgroundenv/lib/python3.14/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:606: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
/Users/athamawardi/miniconda3/envs/playgroundenv/lib/python3.14/site-packages/scipy/sparse/linalg/_matfuncs.py:76: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  Ainv = spsolve(A, I)


: 

## Model comparison (residual Moran, key coefficients)

In [ ]:
# Assemble comparison: baseline residual Moran (M4), SLX post Moran, SDM post Moran (optional), SEM lambda

# Ensure moran_df is available (load from CSV if not in memory after kernel restart)
try:
    moran_df
except NameError:
    _moran_csv = out_path("tables", "moran_results.csv")
    if os.path.exists(_moran_csv):
        moran_df = pd.read_csv(_moran_csv)
        print("(Loaded moran_df from CSV)")
    else:
        moran_df = pd.DataFrame(columns=["sample", "outcome", "exposure", "target", "moran_i", "p_value"])
        print("WARNING: moran_df not available — run M4 cell first")

comparison = []
# From M4
for _, row in moran_df.iterrows():
    if row["target"] == "uhat_baseline":
        comparison.append({
            "sample": row["sample"], "outcome": row["outcome"], "exposure": row["exposure"],
            "model": "baseline", "metric": "moran_i", "value": row["moran_i"],
        })
        comparison.append({
            "sample": row["sample"], "outcome": row["outcome"], "exposure": row["exposure"],
            "model": "baseline", "metric": "moran_p", "value": row["p_value"],
        })
# From M5
for _, row in pd.read_csv(out_path("tables", "slx_post_moran.csv")).iterrows():
    comparison.append({
        "sample": row["sample"], "outcome": row["outcome"], "exposure": row["exposure"],
        "model": "SLX", "metric": "moran_i", "value": row["moran_i"],
    })
    comparison.append({
        "sample": row["sample"], "outcome": row["outcome"], "exposure": row["exposure"],
        "model": "SLX", "metric": "moran_p", "value": row["p_value"],
    })
comp_df = pd.DataFrame(comparison)
comp_df.to_csv(out_path("tables", "model_comparison_moran.csv"), index=False)
print(comp_df.head(20).to_string())
print("\nFull comparison saved to Output/tsls/tables/model_comparison_moran.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/athamawardi/Desktop/Research-Projects/PSE_Pertamina/outputs/tables/slx_post_moran.csv'

## Goodness of Fit — RMSE, R², MAE, AIC, BIC across all models

Compare **Baseline IV2SLS** (from Stata village-values), **M5 SLX**, **M6 SDM (GM_Lag)**, and **M7 SEM (GM_Error)** using standard GOF metrics:

- **RMSE** — Root Mean Square Error (lower = better)
- **MAE** — Mean Absolute Error (lower = better)
- **R² / pseudo-R²** — Variance explained (higher = better)
- **Adj. R²** — Adjusted R² (OLS-based models only)
- **AIC / BIC** — Information Criteria (lower = better, OLS only)
- **Log-Lik** — Log-Likelihood (OLS only)
- **σ²** — Error variance estimate
- **Moran I (resid)** — Residual spatial autocorrelation (closer to 0 = better)
- **N** — Sample size

In [ ]:
# ============================================================
# Goodness-of-Fit computation across all models
# Baseline IV2SLS (Stata), M5 SLX, M6 SDM (GM_Lag), M7 SEM (GM_Error)
# ============================================================

import warnings
warnings.filterwarnings("ignore")

# Ensure moran_df is available (load from CSV if not in memory after kernel restart)
try:
    moran_df
except NameError:
    _moran_csv = out_path("tables", "moran_results.csv")
    if os.path.exists(_moran_csv):
        moran_df = pd.read_csv(_moran_csv)
        print("(Loaded moran_df from CSV)")
    else:
        moran_df = pd.DataFrame(columns=["sample", "outcome", "exposure", "target", "moran_i", "p_value"])
        print("WARNING: moran_df not available — run M4 cell first for Moran I values")

gof_rows = []

# ------------------------------------------------------------------
# Helper: compute standard GOF metrics from y and residuals/predicted
# ------------------------------------------------------------------
def compute_gof(y_true, y_pred, model_name, sample, exposure, outcome, n_params=None, extra=None):
    """Compute RMSE, MAE, R², and optionally AIC/BIC from arrays."""
    y_true = np.asarray(y_true, dtype=float).ravel()
    y_pred = np.asarray(y_pred, dtype=float).ravel()
    resid = y_true - y_pred
    n = len(y_true)

    rmse = np.sqrt(np.mean(resid ** 2))
    mae = np.mean(np.abs(resid))
    ss_res = np.sum(resid ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    r2 = 1.0 - ss_res / ss_tot if ss_tot > 0 else np.nan
    sigma2 = ss_res / n

    row = {
        "sample": sample, "exposure": exposure, "outcome": outcome,
        "model": model_name, "N": n,
        "RMSE": rmse, "MAE": mae, "R2": r2, "sigma2": sigma2,
    }

    # Adjusted R², AIC, BIC, Log-Lik (meaningful for OLS-based)
    if n_params is not None and n_params > 0:
        k = n_params
        adj_r2 = 1.0 - (1.0 - r2) * (n - 1) / (n - k - 1) if n > k + 1 else np.nan
        # Log-likelihood under normality: -n/2 * ln(2*pi*sigma2) - n/2
        ll = -n / 2.0 * (np.log(2 * np.pi * sigma2) + 1) if sigma2 > 0 else np.nan
        aic = 2 * k - 2 * ll if not np.isnan(ll) else np.nan
        bic = k * np.log(n) - 2 * ll if not np.isnan(ll) else np.nan
        row.update({"adj_R2": adj_r2, "AIC": aic, "BIC": bic, "LogLik": ll})
    else:
        row.update({"adj_R2": np.nan, "AIC": np.nan, "BIC": np.nan, "LogLik": np.nan})

    # Merge any extra fields (e.g. Moran I)
    if extra:
        row.update(extra)
    return row


# ==================================================================
# 1. BASELINE IV2SLS (from Stata village_values)
#    y_actual = W_hat + uhat  →  y_pred = W_hat  →  residual = uhat
# ==================================================================
print("=" * 70)
print("1. Baseline IV2SLS GOF (from Stata village-values)")
print("=" * 70)

for exposure in EXPOSURES:
    for v_suffix in ["v1"]:
        key = (exposure, v_suffix)
        if key not in BASELINE_VALUES:
            continue
        vv = BASELINE_VALUES[key].copy()
        for sample in SAMPLES:
            for outcome in OUTCOMES:
                sub = vv[(vv["sample"] == sample) & (vv["outcome"] == outcome) &
                         (vv["spec"] == BASELINE_SPEC)].copy()
                if len(sub) < 10:
                    continue
                sub["uhat"] = pd.to_numeric(sub["uhat"], errors="coerce")
                sub["W_hat"] = pd.to_numeric(sub["W_hat"], errors="coerce")
                sub = sub.dropna(subset=["uhat", "W_hat"])
                if len(sub) < 10:
                    continue
                y_actual = sub["W_hat"].values + sub["uhat"].values
                y_pred = sub["W_hat"].values

                # Moran I on baseline residuals (reuse from M4 if available)
                moran_val = np.nan
                m4_match = moran_df[(moran_df["sample"] == sample) &
                                    (moran_df["outcome"] == outcome) &
                                    (moran_df["exposure"] == exposure) &
                                    (moran_df["target"] == "uhat_baseline")]
                if len(m4_match) > 0:
                    moran_val = m4_match.iloc[0]["moran_i"]

                row = compute_gof(y_actual, y_pred, "Baseline_IV2SLS", sample, exposure, outcome,
                                  extra={"Moran_I_resid": moran_val})
                gof_rows.append(row)

baseline_gof = pd.DataFrame([r for r in gof_rows if r["model"] == "Baseline_IV2SLS"])
print(baseline_gof[["sample", "exposure", "outcome", "N", "RMSE", "MAE", "R2"]].to_string(index=False))


# ==================================================================
# 2. M5 — SLX (re-run to capture full GOF)
# ==================================================================
print("\n" + "=" * 70)
print("2. M5 SLX GOF")
print("=" * 70)

for sample in SAMPLES:
    for exposure in EXPOSURES:
        for outcome in OUTCOMES:
            run, _ = build_run_df(sample, exposure, outcome)
            if run is None or len(run) < 20:
                continue
            W = build_W_for_run(run)
            if W is None:
                continue
            try:
                mod, resid, exog = run_slx(run, W, X_COLS)
                y_true = run["y"].values
                y_pred = y_true - resid  # fitted = y - residual

                # Moran I on SLX residuals
                mi = Moran(resid, W, permutations=999)

                # n_params: const + exog
                k = len(exog) + 1  # +1 for constant

                row = compute_gof(y_true, y_pred, "M5_SLX", sample, exposure, outcome,
                                  n_params=k,
                                  extra={
                                      "Moran_I_resid": mi.I,
                                      "Moran_p_resid": mi.p_sim,
                                      # Also pull from statsmodels directly
                                      "sm_R2": mod.rsquared,
                                      "sm_adj_R2": mod.rsquared_adj,
                                      "sm_AIC": mod.aic,
                                      "sm_BIC": mod.bic,
                                      "sm_LogLik": mod.llf,
                                      "F_stat": mod.fvalue,
                                      "F_pvalue": mod.f_pvalue,
                                  })
                gof_rows.append(row)
            except Exception as e:
                print(f"  SLX skip: {sample}/{exposure}/{outcome} — {e}")

slx_gof = pd.DataFrame([r for r in gof_rows if r["model"] == "M5_SLX"])
print(slx_gof[["sample", "exposure", "outcome", "N", "RMSE", "MAE", "R2", "sm_AIC", "sm_BIC", "Moran_I_resid"]].to_string(index=False))


# ==================================================================
# 3. M6 — SDM / GM_Lag (re-run to capture GOF from spreg object)
# ==================================================================
print("\n" + "=" * 70)
print("3. M6 SDM (GM_Lag) GOF")
print("=" * 70)

for sample in SAMPLES:
    for exposure in EXPOSURES:
        for outcome in OUTCOMES:
            run, _ = build_run_df(sample, exposure, outcome)
            if run is None or len(run) < 20:
                continue
            W = build_W_for_run(run)
            if W is None:
                continue
            try:
                model, betas, run_dm = run_sdm_iv(run, W, INSTRUMENTS, X_COLS)
                if model is None:
                    continue

                y_true = np.asarray(run_dm["y"], dtype=np.float64).ravel()
                resid = np.asarray(model.u).ravel()
                y_pred = np.asarray(model.predy).ravel()

                # Moran I on SDM residuals
                mi = Moran(resid, W, permutations=999)

                # spreg pseudo-R²
                pr2 = getattr(model, "pr2", np.nan)
                pr2_e = getattr(model, "pr2_e", np.nan)
                sig2 = getattr(model, "sig2", np.nan)
                rho = getattr(model, "rho", np.nan)

                k = model.k if hasattr(model, "k") else np.nan
                n = model.n if hasattr(model, "n") else len(y_true)

                row = compute_gof(y_true, y_pred, "M6_SDM", sample, exposure, outcome,
                                  n_params=int(k) if not np.isnan(k) else None,
                                  extra={
                                      "Moran_I_resid": mi.I,
                                      "Moran_p_resid": mi.p_sim,
                                      "pseudo_R2": pr2,
                                      "pseudo_R2_e": pr2_e,
                                      "rho": rho,
                                      "spreg_sig2": sig2,
                                  })
                gof_rows.append(row)
            except Exception as e:
                print(f"  SDM skip: {sample}/{exposure}/{outcome} — {e}")

sdm_gof = pd.DataFrame([r for r in gof_rows if r["model"] == "M6_SDM"])
if len(sdm_gof) > 0:
    print(sdm_gof[["sample", "exposure", "outcome", "N", "RMSE", "MAE", "R2", "pseudo_R2", "rho", "Moran_I_resid"]].to_string(index=False))
else:
    print("  (no SDM results)")


# ==================================================================
# 4. M7 — SEM / GM_Error (re-run to capture GOF)
# ==================================================================
print("\n" + "=" * 70)
print("4. M7 SEM (GM_Error) GOF")
print("=" * 70)

for sample in SAMPLES:
    run_full = DATASETS[sample].copy()
    run_full = add_ring_exposure(run_full)
    for outcome in OUTCOMES:
        if outcome not in run_full.columns:
            continue
        run_full["y"] = run_full[outcome]
        req = ["iddesa", FE_KEY, "lat_v", "lon_v", "y"] + RING_COLS + X_COLS
        req = [c for c in req if c in run_full.columns]
        r = run_full[req].dropna(how="any")
        if len(r) < 20:
            continue
        W = build_W_for_run(r)
        if W is None:
            continue
        r = demean_by(r, FE_KEY, ["y"] + RING_COLS + [c for c in X_COLS if c in r.columns])
        exog = [c for c in RING_COLS + X_COLS if c in r.columns and r[c].notna().all() and pd.api.types.is_numeric_dtype(r[c])]
        if len(exog) < 2:
            continue
        X = np.asarray(r[exog], dtype=np.float64)
        y = np.asarray(r["y"], dtype=np.float64)
        try:
            model = spreg.GM_Error(y, X, w=W, name_x=exog)
            resid = np.asarray(model.u).ravel()
            y_pred = np.asarray(model.predy).ravel()

            # Moran I on SEM residuals
            mi = Moran(resid, W, permutations=999)

            pr2 = getattr(model, "pr2", np.nan)
            pr2_e = getattr(model, "pr2_e", np.nan)
            sig2 = getattr(model, "sig2", np.nan)
            lam = float(np.asarray(model.betas).flat[-1])

            k = len(exog) + 1  # exog + lambda
            n = len(y)

            # Exposure label: "ring" since M7 uses ring terms not a single exposure
            row = compute_gof(y, y_pred, "M7_SEM", sample, "ring", outcome,
                              n_params=k,
                              extra={
                                  "Moran_I_resid": mi.I,
                                  "Moran_p_resid": mi.p_sim,
                                  "pseudo_R2": pr2,
                                  "pseudo_R2_e": pr2_e,
                                  "lambda": lam,
                                  "spreg_sig2": sig2,
                              })
            gof_rows.append(row)
        except Exception as e:
            print(f"  SEM skip: {sample}/{outcome} — {e}")

sem_gof = pd.DataFrame([r for r in gof_rows if r["model"] == "M7_SEM"])
if len(sem_gof) > 0:
    print(sem_gof[["sample", "outcome", "N", "RMSE", "MAE", "R2", "pseudo_R2", "lambda", "Moran_I_resid"]].to_string(index=False))
else:
    print("  (no SEM results)")


# ==================================================================
# 5. Full comparison table
# ==================================================================
print("\n" + "=" * 70)
print("FULL GOODNESS-OF-FIT COMPARISON TABLE")
print("=" * 70)

gof_df = pd.DataFrame(gof_rows)

# Core columns for display
core_cols = ["model", "sample", "exposure", "outcome", "N", "RMSE", "MAE", "R2", "sigma2"]
optional_cols = ["adj_R2", "AIC", "BIC", "LogLik", "pseudo_R2", "rho", "lambda",
                 "Moran_I_resid", "Moran_p_resid", "F_stat"]
display_cols = core_cols + [c for c in optional_cols if c in gof_df.columns]
display_cols = [c for c in display_cols if c in gof_df.columns]

print(gof_df[display_cols].to_string(index=False))

# Save to CSV
gof_df.to_csv(out_path("tables", "goodness_of_fit_all_models.csv"), index=False)
print(f"\nSaved: Output/tsls/tables/goodness_of_fit_all_models.csv  ({len(gof_df)} rows)")

1. Baseline IV2SLS GOF (from Stata village-values)
sample exposure  outcome     N     RMSE      MAE       R2
 COAST  A_v_log  lnpov_z 12966 0.694043 0.529705 0.188506
 COAST  A_v_log Wbasic_z 12966 0.438396 0.342761 0.191638
   ALL  A_v_log  lnpov_z 84276 0.651414 0.483291 0.110459
   ALL  A_v_log Wbasic_z 84276 0.424332 0.326337 0.090292
 COAST  Ker5_w1  lnpov_z 12966 0.720343 0.550410 0.084113
 COAST  Ker5_w1 Wbasic_z 12966 0.453430 0.352476 0.091058
   ALL  Ker5_w1  lnpov_z 84276 0.659338 0.488932 0.020050
   ALL  Ker5_w1 Wbasic_z 84276 0.429031 0.329834 0.013154
 COAST Ker10_w1  lnpov_z 12966 0.714318 0.546152 0.160156
 COAST Ker10_w1 Wbasic_z 12966 0.450209 0.351355 0.166300
   ALL Ker10_w1  lnpov_z 84276 0.658657 0.489105 0.039231
   ALL Ker10_w1 Wbasic_z 84276 0.428916 0.330240 0.027777

2. M5 SLX GOF
  SLX skip: COAST/A_v_log/lnpov_z — name 'run_slx' is not defined
  SLX skip: COAST/A_v_log/Wbasic_z — name 'run_slx' is not defined
  SLX skip: COAST/Ker5_w1/lnpov_z — name 'run_s

KeyError: "None of [Index(['sample', 'exposure', 'outcome', 'N', 'RMSE', 'MAE', 'R2', 'sm_AIC',\n       'sm_BIC', 'Moran_I_resid'],\n      dtype='object')] are in the [columns]"

In [ ]:
# ============================================================
# Summary pivot tables & bar chart comparison
# ============================================================

# Ensure gof_df is available (load from CSV if not in memory after kernel restart)
try:
    gof_df
except NameError:
    _gof_csv = out_path("tables", "goodness_of_fit_all_models.csv")
    if os.path.exists(_gof_csv):
        gof_df = pd.read_csv(_gof_csv)
        print("(Loaded gof_df from CSV)")
    else:
        raise NameError("gof_df not available and no CSV found. Run the GOF cell above first.")

# --- Pivot: RMSE by model × (sample, exposure, outcome) ---
print("=" * 70)
print("RMSE by Model (lower is better)")
print("=" * 70)
rmse_pivot = gof_df.pivot_table(
    index=["sample", "exposure", "outcome"],
    columns="model",
    values="RMSE",
    aggfunc="first"
).round(4)
print(rmse_pivot.to_string())

print("\n" + "=" * 70)
print("R² by Model (higher is better)")
print("=" * 70)
r2_pivot = gof_df.pivot_table(
    index=["sample", "exposure", "outcome"],
    columns="model",
    values="R2",
    aggfunc="first"
).round(4)
print(r2_pivot.to_string())

print("\n" + "=" * 70)
print("Moran I (residuals) by Model (closer to 0 → less residual spatial autocorrelation)")
print("=" * 70)
if "Moran_I_resid" in gof_df.columns:
    moran_pivot = gof_df.pivot_table(
        index=["sample", "exposure", "outcome"],
        columns="model",
        values="Moran_I_resid",
        aggfunc="first"
    ).round(4)
    print(moran_pivot.to_string())

# --- Bar chart: average RMSE per model ---
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Panel 1: Average RMSE
avg_rmse = gof_df.groupby("model")["RMSE"].mean().sort_values()
avg_rmse.plot.barh(ax=axes[0], color=["#2196F3", "#4CAF50", "#FF9800", "#F44336"][:len(avg_rmse)])
axes[0].set_title("Mean RMSE by Model (lower = better)", fontsize=12)
axes[0].set_xlabel("RMSE")
for i, (v, name) in enumerate(zip(avg_rmse.values, avg_rmse.index)):
    axes[0].text(v + 0.001, i, f"{v:.4f}", va="center", fontsize=9)

# Panel 2: Average R²
avg_r2 = gof_df.groupby("model")["R2"].mean().sort_values(ascending=False)
avg_r2.plot.barh(ax=axes[1], color=["#2196F3", "#4CAF50", "#FF9800", "#F44336"][:len(avg_r2)])
axes[1].set_title("Mean R² by Model (higher = better)", fontsize=12)
axes[1].set_xlabel("R²")
for i, (v, name) in enumerate(zip(avg_r2.values, avg_r2.index)):
    axes[1].text(v + 0.001, i, f"{v:.4f}", va="center", fontsize=9)

# Panel 3: Average Moran I (residuals)
if "Moran_I_resid" in gof_df.columns:
    avg_moran = gof_df.groupby("model")["Moran_I_resid"].mean().sort_values()
    avg_moran.plot.barh(ax=axes[2], color=["#2196F3", "#4CAF50", "#FF9800", "#F44336"][:len(avg_moran)])
    axes[2].set_title("Mean Moran I (resid) by Model\n(closer to 0 = less spatial autocorrelation)", fontsize=11)
    axes[2].set_xlabel("Moran I")
    for i, (v, name) in enumerate(zip(avg_moran.values, avg_moran.index)):
        axes[2].text(v + 0.001, i, f"{v:.4f}", va="center", fontsize=9)

plt.tight_layout()
plt.savefig(out_path("figures", "gof_comparison_barplot.png"), dpi=150, bbox_inches="tight")
plt.show()
print("\nSaved: Output/tsls/figures/gof_comparison_barplot.png")

# --- Detailed per-run comparison plot: RMSE across all runs ---
fig, ax = plt.subplots(figsize=(14, max(6, len(gof_df) * 0.25)))
models_ordered = ["Baseline_IV2SLS", "M5_SLX", "M6_SDM", "M7_SEM"]
colors = {"Baseline_IV2SLS": "#9E9E9E", "M5_SLX": "#2196F3", "M6_SDM": "#4CAF50", "M7_SEM": "#FF9800"}
gof_df["run_label"] = gof_df["sample"] + " | " + gof_df["exposure"] + " | " + gof_df["outcome"]
for m in models_ordered:
    sub = gof_df[gof_df["model"] == m].sort_values("run_label")
    if len(sub) > 0:
        ax.barh(sub["run_label"] + f"  ({m})", sub["RMSE"], label=m,
                color=colors.get(m, "#607D8B"), alpha=0.85, height=0.7)
ax.set_xlabel("RMSE", fontsize=12)
ax.set_title("RMSE per Model × Run", fontsize=13)
ax.legend(loc="lower right")
plt.tight_layout()
plt.savefig(out_path("figures", "gof_rmse_per_run.png"), dpi=150, bbox_inches="tight")
plt.show()
print("Saved: Output/tsls/figures/gof_rmse_per_run.png")

RMSE by Model (lower is better)


NameError: name 'gof_df' is not defined